In [3]:
import pandas as pd
from chroma_db import ChromaDB

In [4]:
data = pd.read_csv('Corpus_v4.csv')
data.head()

,link,text,pages,is_markdown
0,https://cbr.ru//support_measures/,Меры защиты финансового рынка Банк России прин...,NaN,False
1,https://cbr.ru//support_measures/,Меры защиты финансового рынка Банк России прин...,NaN,False
2,https://cbr.ru//reception/,Интернет-приемная Хотите обратиться в Банк Рос...,NaN,False
3,https://cbr.ru//reception/,Интернет-приемная Хотите обратиться в Банк Рос...,NaN,False
4,https://cbr.ru//support_measures/,Меры защиты финансового рынка Банк России прин...,NaN,False


In [5]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('punkt')

c:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Никита\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
def get_nltk_chunks(text, chunk_size=200, chunk_overlap=100, language="russian"):
    words = word_tokenize(text, language=language)

    chunks = []

    end_index = chunk_size
    while end_index - chunk_size < len(words):
        sentence = ''
        for word in words[end_index - chunk_size:end_index]:
            if word in ',.?!:)»' or (sentence and sentence[-1] in '«('):
                sentence += word
            else:
                sentence += " " + word

        chunks.append(sentence)
        end_index += chunk_size - chunk_overlap

    return chunks

In [17]:
# for chunk in get_nltk_chunks(data.text[319], chunk_size = 200, chunk_overlap = 20):
#     print(len(chunk), chunk)
#     print()

In [7]:
chroma = ChromaDB('84.201.178.246', port=8000)

In [1]:
import logging

logging.basicConfig(level=logging.INFO, filename='db.log', filemode="a",
                       format="%(asctime)s %(levelname)s %(message)s")

In [20]:
for index, row in data[:10000].iterrows(): 
    try:
        if index % 25 == 0:
            logging.info(f'{index} done')
        link = row.link
        text = row.text
        pages = row.pages
        is_markdown = row.is_markdown
        if is_markdown:
            chunks = get_nltk_chunks(text, chunk_size = 2000, chunk_overlap = 100)
        else:
            chunks = get_nltk_chunks(text, chunk_size = 200, chunk_overlap = 20)
        chroma.insert_document(index, chunks, link, is_markdown, pages)
    except Exception as e:
        logging.error(index)